# Pull .md and extract links to add to metadata

In [16]:
from llama_index.readers.github import GithubRepositoryReader, GithubClient
import os
import re

In [10]:
import nest_asyncio
nest_asyncio.apply()

In [8]:
github_token = ""

In [ ]:
client = github_client = GithubClient(github_token=github_token, verbose=False)

reader = GithubRepositoryReader(
    github_client=github_client,
    owner="Infineon",
    repo="mtb-example-btsdk-low-power-20819",
    use_parser=False,
    verbose=True,
    filter_directories=(
        ["deps", ".gitignore", "Makefile", "LICENSE"],
        GithubRepositoryReader.FilterType.EXCLUDE,
    ),
    filter_file_extensions=(
        [
            ".png",
            ".jpg",
            ".jpeg",
            ".gif",
            ".svg",
            ".ico",
            "json",
            ".ipynb",
            ".c",
            ".h",
            ".gitignore",
            ".cybt",
            ".mtb"
        ],
        GithubRepositoryReader.FilterType.EXCLUDE,
    ),
)

documents = reader.load_data(branch="master")

In [14]:
for doc in documents:
    print(doc.metadata)

{'file_path': 'README.md', 'file_name': 'README.md', 'url': 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/README.md'}


In [ ]:
documents[0].text


In [20]:
links = re.findall(r'https?://[^\s)"]+(?=[)\s,."\'!])?', documents[0].text)

37


In [34]:
import requests
text=[]
def download_pdf(url, filename):
    response = requests.get(url, timeout=5)
    with open(f"{filename}.pdf", 'wb') as f:
        f.write(response.content)
    print(f"Downloaded PDF: {filename}.pdf")

def save_text(url, text_array):
    response = requests.get(url, timeout=5)
    text.append(response.text)  # Append the text to the array
    print("Text added to array")

def process_links(links):
    for url in links:
        try:
            response = requests.head(url, allow_redirects=True, timeout=5)  # Use head to get headers
            content_type = response.headers.get('Content-Type', '')

            if 'pdf' in content_type:
                print(f"PDF found: {url}")
                download_pdf(url, url.split('/')[-1].split('.')[0])  # Save with filename without extension
            elif 'text' in content_type or 'html' in content_type:
                print(f"Text or webpage found: {url}")
                save_text(url, url.split('/')[-1].split('.')[0])  # Save with filename without extension
            else:
                print(f"Other content type ({content_type}) for URL: {url}")
        except requests.RequestException as e:
            print(f"Error fetching {url}: {e}")

# Example links
# links = ['https://example.com/file.pdf', 'https://example.com/page', 'https://example.com/info.txt']
links = ['https://ttssh2.osdn.jp/index.html.en', 'https://www.infineon.com/ModusToolboxProjectCreator', 'https://www.infineon.com/ModusToolboxLibraryManager', 'https://www.infineon.com/cms/en/design-support/software/free-and-open-source-software-foss/>.', 'https://www.infineon.com/ModusToolboxProjectCreator', 'https://www.infineon.com/dgdl/Infineon-ModusToolbox_2.4_User_Guide-Software-v01_']

process_links(links)


Error fetching https://ttssh2.osdn.jp/index.html.en: HTTPSConnectionPool(host='ttssh2.osdn.jp', port=443): Max retries exceeded with url: /index.html.en (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x143567100>, 'Connection to ttssh2.osdn.jp timed out. (connect timeout=5)'))
PDF found: https://www.infineon.com/ModusToolboxProjectCreator
Downloaded PDF: ModusToolboxProjectCreator.pdf
PDF found: https://www.infineon.com/ModusToolboxLibraryManager
Downloaded PDF: ModusToolboxLibraryManager.pdf
Text or webpage found: https://www.infineon.com/cms/en/design-support/software/free-and-open-source-software-foss/>.
Text added to array
PDF found: https://www.infineon.com/ModusToolboxProjectCreator
Downloaded PDF: ModusToolboxProjectCreator.pdf
Text or webpage found: https://www.infineon.com/dgdl/Infineon-ModusToolbox_2.4_User_Guide-Software-v01_
Text added to array


In [ ]:
text[1]

In [ ]:
links_main = ['https://ttssh2.osdn.jp/index.html.en', 'https://www.infineon.com/ModusToolboxProjectCreator', 'https://www.infineon.com/ModusToolboxLibraryManager', 'https://www.infineon.com/cms/en/design-support/software/free-and-open-source-software-foss/>.', 'https://www.infineon.com/ModusToolboxProjectCreator', 'https://www.infineon.com/dgdl/Infineon-ModusToolbox_2.4_User_Guide-Software-v01_', 'https://ttssh2.osdn.jp/index.html.en', 'https://www.infineon.com/ModusToolboxProjectCreator', 'https://www.infineon.com/ModusToolboxLibraryManager', 'https://www.infineon.com/cms/en/design-support/software/free-and-open-source-software-foss/>.', 'https://www.infineon.com/ModusToolboxProjectCreator', 'https://www.infineon.com/dgdl/Infineon-ModusToolbox_2.4_User_Guide-Software-v01_', 'https://ttssh2.osdn.jp/index.html.en', 'https://www.infineon.com/ModusToolboxProjectCreator', 'https://www.infineon.com/ModusToolboxLibraryManager', 'https://www.infineon.com/cms/en/design-support/software/free-and-open-source-software-foss/>.', 'https://www.infineon.com/ModusToolboxProjectCreator', 'https://www.infineon.com/dgdl/Infineon-ModusToolbox_2.4_User_Guide-Software-v01_']

links_small= ['https://ttssh2.osdn.jp/index.html.en', 'https://www.infineon.com/ModusToolboxProjectCreator', 'https://www.infineon.com/ModusToolboxLibraryManager', 'https://www.infineon.com/cms/en/design-support/software/free-and-open-source-software-foss/>.', 'https://www.infineon.com/ModusToolboxProjectCreator', 'https://www.infineon.com/dgdl/Infineon-ModusToolbox_2.4_User_Guide-Software-v01_']

In [32]:
from langchain_openai import OpenAIEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
# Initialize OpenAI Embeddings
embeddings = OpenAIEmbeddings()

# Example list of links
links = [
    "Guide to deploying AWS services effectively.",
    "How to deploy an edge system in AWS.",
    "AWS security best practices for developers.",
    "Introduction to edge computing and AWS solutions.",
    "Using Docker with AWS for seamless deployment."
]

# # Embed the links
link_embeddings = embeddings.embed_documents(links)
print('ch')
# Define the query (your question)
question = "How to deploy an edge in AWS?"

# Embed the query
query_embedding = embeddings.embed_query(question)

# Calculate cosine similarity between the query and all links
similarity_scores = cosine_similarity([query_embedding], link_embeddings)[0]

# Get the indices of the top 5 most similar links
top_5_indices = np.argsort(similarity_scores)[-5:][::-1]

# Return the top 5 links
top_5_links = [(links[i], similarity_scores[i]) for i in top_5_indices]

# Display the best 5 links
# for idx, (link, score) in enumerate(top_5_links, 1):
#     print(f"{idx}. {link} (Score: {score:.4f})")
print(top_5_links)

ch
[('How to deploy an edge system in AWS.', 0.9732275149479189), ('Introduction to edge computing and AWS solutions.', 0.8868563293776737), ('Guide to deploying AWS services effectively.', 0.8532092459198723), ('Using Docker with AWS for seamless deployment.', 0.832225674598413), ('AWS security best practices for developers.', 0.799077206100842)]
